In [1]:
from IPython.display import clear_output
!wget https://disk.karelia.pro/djnSMfz/Трек%201.%20Онежское%20озеро.tgz?attachment
import os
try:
    os.mkdir("./data")
except: pass
!tar -xvzf "Трек 1. Онежское озеро.tgz?attachment" -C ./data
clear_output()

In [5]:
from google.colab import drive
drive.mount('/content/drive')
!pip install catboost

Mounted at /content/drive
     |████████████████████████████████| 65.7MB 50kB/s 


In [6]:
# ваша/наша папка
folder = "Ситилинк Хакатон/Для организаторов/Ситилинк Хакатон/Рабочее решение"
# "./drive/My Drive/" обязательно должен быть
# по нему мы обращаемся к моделям
path = f"./drive/My Drive/{folder}/"
print(path)

./drive/My Drive/Ситилинк Хакатон/Для организаторов/Ситилинк Хакатон/Рабочее решение/


In [11]:
import os
os.listdir(path)

['2016-2021forecast.csv',
 'joined.csv',
 'info.ipynb',
 'joined_last.csv',
 'joined_new.csv',
 'ModernRegressor_train.ipynb',
 'random',
 'torch_train.ipynb',
 'baseline_v1.1.ipynb',
 'Models',
 'baseline_v1_2.ipynb',
 'ModernClassifier_train.ipynb']

In [7]:
import os
from torchvision import transforms
import pickle
import pandas as pd
import torch
from tqdm.notebook import tqdm
from PIL import Image
import numpy as np
 
class LakeClassifier():
    def __init__(self, models_path):
        self.device = torch.device('cuda' if 
                                   torch.cuda.is_available() else 'cpu')
        self.network = torch.load(os.path.join(models_path, "network.pth")).to(self.device)
        self.transform = transforms.ToTensor()
        self.size = (128, 128)
    
    def predict_network(self, path):
        predictions = []
        for filename in tqdm(os.listdir(path)):
            with Image.open(os.path.join(path, filename)) as image:
                image = image.resize(self.size)
            X = self.transform(image)
            X = X.view(1, *X.size()).to(self.device)
            prediction = self.network(X).argmax(dim=1).item()
            predictions.append([filename, prediction])
        return predictions

In [13]:
model_nn = LakeClassifier(os.path.join(path, "Models"))

In [14]:
predictions = model_nn.predict_network("data")

In [15]:
import pandas as pd

# переводим дату в числа
def date_to_data(frame, holidays=None,
                 drop=False, name="date", suffix=""):
    assert name in frame, "Нужна колонка с датой"
    date = frame[name].dt

    info = pd.concat([date.days_in_month,
                     date.month,
                     date.quarter,
                     date.month % 12 // 3 + 1], axis=1)
    
    info.columns = ["days_in_month", "month",
                    "quarter", "season"]

    temp = pd.concat([date.isocalendar(),
                      info], axis=1)

    temp.columns = temp.columns + suffix
    frame = pd.concat([frame, temp], axis=1)
    if drop:
        return frame.drop([name], axis=1)
    return frame

df = pd.read_csv(path + "joined_last.csv", index_col="Unnamed: 0")
df["date"] = pd.to_datetime(df["name"].apply(lambda x: x[:-4]))
df = date_to_data(df, name="date", drop=False)

In [16]:
predictions_df = pd.DataFrame(predictions, columns=["filename", "prediction"])

In [17]:
frame = df.merge(predictions_df, left_on="name", right_on="filename", how="left")

In [18]:
names = ['hour', 'year', 'week', 'day',
        'days_in_month', 'month', 'quarter', 'season',
        'prediction']

In [19]:
from sklearn.model_selection import train_test_split

def train_test_val_split(X, y,
                         test_size=0.3, val_size=0.3,
                         random_state=42, shuffle=True):
    """
    separator for train, test, validation
    """
    X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                        train_size=1 - (test_size + val_size),
                                                        random_state=random_state, shuffle=shuffle)
    X_val, X_test, y_val, y_test = train_test_split(X_test, y_test,
                                                    test_size=test_size / (test_size + val_size),
                                                    random_state=random_state, shuffle=shuffle)
    return X_train, y_train, X_test, y_test, X_val, y_val

X = frame[names].astype(np.float32)
y = frame["y"].astype(np.int32)

X_train, y_train, X_test, y_test, X_val, y_val = train_test_val_split(X, y,
                                                                      test_size=0.3,
                                                                      val_size=0.3,
                                                                      shuffle=False)

In [20]:
from catboost import CatBoostClassifier

clf = CatBoostClassifier(random_state=42,
                         verbose=1,
                         iterations=100,
                         grow_policy="Depthwise",
                         loss_function="CrossEntropy",
                         task_type="GPU")

# clf.fit(X_train, y_train)
clf.fit(X, y)

0:	learn: 0.6650123	total: 9.3ms	remaining: 920ms
1:	learn: 0.6384172	total: 17.3ms	remaining: 849ms
2:	learn: 0.6132573	total: 33.5ms	remaining: 1.08s
3:	learn: 0.5895292	total: 41ms	remaining: 983ms
4:	learn: 0.5671185	total: 54.6ms	remaining: 1.04s
5:	learn: 0.5458225	total: 62.2ms	remaining: 974ms
6:	learn: 0.5256385	total: 69.9ms	remaining: 929ms
7:	learn: 0.5064021	total: 77.4ms	remaining: 890ms
8:	learn: 0.4881125	total: 85.2ms	remaining: 862ms
9:	learn: 0.4707204	total: 92.6ms	remaining: 833ms
10:	learn: 0.4542065	total: 104ms	remaining: 838ms
11:	learn: 0.4383533	total: 111ms	remaining: 815ms
12:	learn: 0.4232055	total: 119ms	remaining: 796ms
13:	learn: 0.4087261	total: 127ms	remaining: 778ms
14:	learn: 0.3949041	total: 134ms	remaining: 761ms
15:	learn: 0.3817529	total: 142ms	remaining: 746ms
16:	learn: 0.3690845	total: 150ms	remaining: 731ms
17:	learn: 0.3569647	total: 157ms	remaining: 716ms
18:	learn: 0.3453775	total: 165ms	remaining: 703ms
19:	learn: 0.3342268	total: 173ms	

In [21]:
# from sklearn.metrics import accuracy_score
# accuracy_score(y_val, clf.predict(X_val))

In [22]:
# from sklearn.metrics import accuracy_score
# accuracy_score(y_test, clf.predict(X_test))

In [23]:
sorted(zip(clf.feature_names_, clf.feature_importances_),
       reverse=True, key=lambda x: x[1])

[('week', 82.21154800373284),
 ('prediction', 7.835093441515018),
 ('year', 7.274947991378629),
 ('days_in_month', 1.3118248414850386),
 ('month', 0.8369297409211234),
 ('day', 0.4372454134139418),
 ('season', 0.06546075805028496),
 ('hour', 0.016777987818721123),
 ('quarter', 0.010171821684406546)]

In [24]:
import pickle
with open(path + "Models/ModernClassifier.pkl", "wb") as f:
    pickle.dump(clf, f)